In [1]:
# Import required  modules
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [2]:
# Voucher code to check
voucher_df = pd.read_csv("Hyperli One Education Voucher codes - Sheet1.csv")
voucher_df.head()

,Voucher Code
0,HYP-SHUPF
1,HYP-68S8K
2,HYP-UA297
3,HYP-AYLE1
4,HYP-NE7OO


In [3]:
# Initialize the driver
driver = webdriver.Chrome("/home/faysal/Documents/utilities/chromedriver")

# Open the link in the browser
driver.get("https://portal.hyperli.com/redeem-voucher")

# Wait for 40 seconds so that user can log in by himself
time.sleep(40)

In [4]:
# Function to get the voucher status text
def get_voucher_status(voucher_code):
    
    # Find the input element tag
    inputElement = driver.find_element_by_id("voucherCode")
    
    # Input the voucher code
    inputElement.send_keys(voucher_code)
    
    #  Send voucher code to the server
    inputElement.submit()
    time.sleep(5)
    
    # Explicitly wait 10 seconds for the element to appear
    try:
        req_text = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.ID, "toast-container"))).text.strip()
        temp_df = pd.DataFrame({
            "voucher_code":voucher_code,
            "status":req_text # Code status text
        }, index=[0])
    except:
        temp_df = pd.DataFrame({
            "voucher_code":voucher_code,
            "status":"na" # If an exception raises
        }, index=[0])
    return temp_df

In [5]:
# Wrap all the function inside main
def main(start, end, file_name):
    final_df = pd.concat(list(map(get_voucher_status, voucher_df["Voucher Code"].iloc[start:end]))).reset_index(drop=True)
    final_df.status = final_df.status.str.split("\n").str[-1].str.strip()
    final_df.to_csv(f"{file_name}_{start}_{end}.csv", index=None)
    return final_df

In [ ]:
# Apply the function
df = main(0, 100, "faysal")
df.head(10)